In [8]:
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
from sklearn.metrics import jaccard_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from scipy.stats import mode

In [9]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

data_file_path = '/Users/asish/Documents/F drive/Future/CSE572_DM/Homework/HW3/kmeans_data/data.csv'
data_df = pd.read_csv(data_file_path)

label_file_path = '/Users/asish/Documents/F drive/Future/CSE572_DM/Homework/HW3/kmeans_data/label.csv'
label_df = pd.read_csv(label_file_path)
cluster_labels = label_df.values

features = data_df.values

scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)


In [13]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity

class KMeans:
    def __init__(self, num_clusters, similarity='euclidean', max_iters=500, tol=1e-4):
        self.num_clusters = num_clusters
        self.similarity = similarity
        self.max_iters = max_iters
        self.tol = tol

    @staticmethod
    def jaccard_similarity(vector_a, vector_b):
        """Compute Jaccard similarity between two vectors."""
        intersection = np.sum(np.minimum(vector_a, vector_b))
        union = np.sum(np.maximum(vector_a, vector_b))
        return intersection / union if union != 0 else 0

    def _compute_distances(self, data, centroids):
        if self.similarity == 'euclidean':
            return pairwise_distances(data, centroids, metric='euclidean')
        elif self.similarity == 'cosine':
            return 1 - cosine_similarity(data, centroids)
        elif self.similarity == 'jaccard':
            distances = np.zeros((len(data), self.num_clusters))
            for i in range(len(data)):
                for j, centroid in enumerate(centroids):
                    distances[i, j] = 1 - self.jaccard_similarity(data[i], centroid)
            return distances

    def fit(self, data):
        # Initialize centroids randomly
        centroids = data[np.random.choice(data.shape[0], self.num_clusters, replace=False)]
        prev_sse = np.inf  # Initialize SSE to infinity for the first iteration
        
        # Iterating until convergence or maximum iterations reached
        for _ in range(self.max_iters):
            # Compute distances from data points to centroids
            distances = self._compute_distances(data, centroids)

            # Assign each data point to the closest centroid
            labels = np.argmin(distances, axis=1)
            
            # Update centroids
            new_centroids = np.zeros((self.num_clusters, data.shape[1]))
            for i in range(self.num_clusters):
                new_centroids[i] = np.mean(data[labels == i], axis=0)
            
            # Calculate SSE
            sse = 0
            for i in range(self.num_clusters):
                sse += np.sum((data[labels == i] - new_centroids[i])**2)
            
            # Check convergence criteria
            if np.linalg.norm(new_centroids - centroids) < self.tol or sse > prev_sse:
                break
            
            prev_sse = sse  # Update SSE for the next iteration
            
            # Update centroids for the next iteration
            centroids = new_centroids
        
        self.centroids = centroids
        self.labels = labels
        self.sse = sse
        self.iterations = _ + 1  # Increment iteration count

        return self.centroids, self.labels, self.sse, self.iterations


In [14]:
k = len(np.unique(cluster_labels[:, 0]))

# Apply k-means with Euclidean similarity
centroids_euclidean, labels_euclidean, sse_euclidean, iterations_euclidean = kmeans(features_standardized, k, similarity='euclidean')

# Apply k-means with Cosine similarity
centroids_cosine, labels_cosine, sse_cosine, iterations_cosine = kmeans(features_standardized, k, similarity='cosine')

# Apply k-means with Jaccard similarity
centroids_jaccard, labels_jaccard, sse_jaccard, iterations_jaccard = kmeans(features_standardized, k, similarity='jaccard')


/var/folders/5g/hkx86931401f03w4p23zrc5w0000gn/T/ipykernel_10901/3921122653.py:28: RuntimeWarning: Mean of empty slice.
  new_centroids = np.array([data[labels == i].mean(axis=0) for i in range(num_clusters)])
/opt/anaconda3/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


In [15]:
# Print the number of iterations taken for convergence
print("Iterations for convergence:")
print("Euclidean:", iterations_euclidean)
print("Cosine:", iterations_cosine)
print("Jaccard:", iterations_jaccard)

Iterations for convergence:
Euclidean: 62
Cosine: 32
Jaccard: 2
